In [1]:
import numpy as np

In [76]:
data  = np.random.rand(6,6)*1000 # some random data

### 3.1 Connectivity

In [77]:
def RBF(x, y, alpha=1):
    """ Gaussian kernel for two vectors x and y. With neighborhood distance alpha.
    
    Arguments:
        - x : First vector
        - y : second vector
        - alpha: neighborhood factor. *Note bigger alpha increases RBF value.
    
    return: Real num in [0, 1]
    """
    return np.exp(-np.linalg.norm(np.array(x) - np.array(y))**2 / alpha)

In [78]:
# calculation check
assert((RBF([0,0], [1,1], 1) - np.exp(-2)) < 0.0001)
assert((RBF([0,0], [0,0], 1) - 1 < 0.0001))

In [87]:
def kernelize(data, k, alpha=1):
    """ Given some data convert it to a kernel matrix using some given kernel.
    
    Arguments:
        data : Data matrix (N, C)
        k: Kernel function
        alpha: neighborhood factor. *Note bigger alpha increases kernel function value.
    """
    pass

### 3.3 Diffusion Distance

In [89]:
# Note theres no square in the diffusion distance
def diffusion_dist(i, j, P):
    """ Computes the diffusion distance between two vectors X_i and X_j.
    
    Arguments:
        i: index for the first data point
        j: index for the second data point
        P: Probability matrix P[i, j] is the probability to go from i to j.
    """
    return np.linalg.norm(P[i, :] - P[j, :])**2

### 3.4 Diffusion Map